# **Tropos (API Feedback Generation)**
#### *This notebook handles set up to Github/Google Drive, model feedback generation, and generates docx files with feedback.*

## **--Set up**

In [1]:
#######################################################################
# Sets up Google Drive, Github, Github branch
# Installs requirements.txt and needed libraries
#######################################################################
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b main origin/main

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet #for chatgpt
!pip install -q google-generativeai --quiet #for gemini
!pip install anthropic httpx --quiet #for claude

/content
Cloning into 'project'...
remote: Enumerating objects: 1598, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1598 (delta 116), reused 92 (delta 92), pack-reused 1373 (from 2)
Receiving objects: 100% (1598/1598), 401.47 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (929/929), done.
Updating files: 100% (166/166), done.
/content/project
fatal: A branch named 'main' already exists.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 

In [ ]:
!python3 ./tropos/app/blocks_ui_starter.py

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://38ab8a8bf150fdf306.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
########################################################################
# Default imports and connecting all API keys
#######################################################################
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
import anthropic
openai_key = userdata.get("conner").strip()
deepseek_key = userdata.get("deepseek").strip()
llama_key = userdata.get("groq").strip()
claude_key = userdata.get("claude").strip()


os.environ["OPENAI_API_KEY"] = openai_key

from openai import OpenAI
client = OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com")

os.environ["LLAMA_API_URL"] = "https://api.groq.com/openai/v1"  # For Ollama
os.environ["LLAMA_API_KEY"] = llama_key  #
os.environ["CLAUDE_API_KEY"] = claude_key

import google.generativeai as genai
genai.configure(api_key=userdata.get("gemini").strip())


## **--Feedback Generation Prep**

In [ ]:
#######################################################################
# Ensure past generated output is cleared
# (comment out if do not want)
#######################################################################
from utils.file_utils import clear_directory_if_exists
output_dir = "/content/project/data/generated_output"
clear_directory_if_exists(output_dir)

✅ All files deleted from generated_output.


In [ ]:
#######################################################################
# Set up paths for feedback engine
#######################################################################
from tropos import test_feedback_console
reqs_file = "./data/raw/Requirements.docx"
examples_dir = "./data/raw/Student_Submissions"
targets_dir = "./data/unmarked_raw"
outputs_dir = "./data/generated_output"
profile_save_path = "./data/generated_profile/instructor_profile.txt"

## **--Instructor Profile Created Feedback Generator**

In [ ]:
# --------------------------------------
# Step 1: Generate Instructor Profile (GPT-4o)
# --------------------------------------

from tropos.models.prompt_builder import generate_full_instructor_profile, load_profile_from_txt

# Paths
examples_dir = "./data/raw/Student_Submissions"
reqs_file = "./data/raw/Requirements.docx"
profile_save_path = "./data/generated_profile/instructor_profile.txt"

batch_size = 7
model_name = "gpt-4o"

# Generate the Instructor Profile
generate_new_profile = True

if generate_new_profile:
    final_profile = generate_full_instructor_profile(
        examples_dir=examples_dir,
        requirements_path=reqs_file,
        batch_size=batch_size,
        model_name=model_name,
        debug=True
    )

    os.makedirs(os.path.dirname(profile_save_path), exist_ok=True)
    with open(profile_save_path, "w", encoding="utf-8") as f:
        f.write(final_profile)

    print(f"📂 Final Instructor Profile saved to {profile_save_path}")
else:
    final_profile = load_profile_from_txt(profile_save_path)
    print("✅ Loaded Instructor Profile successfully!")


📦 Loaded 39 examples
🔹 Processing Batch 1 (7 examples)
📝 Mini Profile 1:
The instructor's feedback habits, tone, priorities, and style can be characterized as follows:

1. **Feedback Habits:**
   - The instructor provides detailed and specific feedback primarily through inline comments and rubric feedback.
   - Feedback is often aimed at clarifying misconceptions and encouraging students to refine their arguments or structure.
   - While there are instances where feedback is not provided, when it is given, it is thorough and focused on guiding the student to a better understanding of the assignment requirements.

2. **Tone:**
   - The tone is constructive and supportive, with an emphasis on helping students improve their work.
   - The instructor uses a direct approach to point out inaccuracies and areas needing improvement but does so in a way that encourages learning and revision.
   - There is a sense of encouragement, as seen in comments that acknowledge positive aspects of the stu

In [ ]:
from tropos import test_feedback_console
from tropos.models.prompt_builder import load_profile_from_txt

profile_save_path = "./data/generated_profile/instructor_profile.txt"
final_profile = load_profile_from_txt(profile_save_path)

test_feedback_console(
    prompt_type="ProfileShot",
    model="gpt-4o",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    output_mode="pretty",
    max_examples=None,
    profile_text=final_profile
)

--- ProfileShot Feedback for Uncommented_Student_1 ---
📝 Instructor Feedback

--- INLINE FEEDBACK (AT LEAST 4 REQUIRED) ---
🗨️  "Unfortunately, the year 2019 is known as the year of quarantine." 👉 – This
sentence could be misleading as the quarantine measures were mainly known to
have started in 2020. Consider rephrasing for clarity.
🗨️  "The mask mandate refers to the authoritative governmental policy of wearing
a mask in certain settings when out in public." 👉 – This definition is clear but
could benefit from additional context, such as explaining which entities enforce
these mandates (e.g., state governments, local authorities).
🗨️  "Statistically, noticeable declines were present with the implementation of
masks in not only county level cases but also death rates." 👉 – This claim needs
to be supported by specific data or sources to strengthen its credibility.
🗨️  "Although there are demands that this is only about politics and social
control, with governmental ownership of the body

## **--Currently Running Models**

In [ ]:
# GPT-4o
test_feedback_console(
    prompt_type="FewShot",
    model="gpt-4o",
    requirements_path=reqs_file,
    example_dir=examples_dir,
    target_dir=targets_dir,
    output_dir=outputs_dir,
    output_mode="pretty",
    max_examples=3
)


--- FewShot Feedback for Uncommented_Student_1 ---
📝 Instructor Feedback

--- INLINE FEEDBACK (AT LEAST 4 REQUIRED) ---

🗨️  "Unfortunately, the year 2019 is known as the year of quarantine." 👉 – This
statement is misleading as quarantine measures were more prominent in 2020.
Consider revising for accuracy.
🗨️  "Although there are many claims, the main one that is often brought up is
the violation of the constitution’s First Amendment." 👉 – This is a strong claim
and would benefit from a specific example or citation to support it.
🗨️  "Statistically, noticeable declines were present with the implementation of
masks in not only county level cases but also death rates." 👉 – Consider
providing specific statistics or a source to strengthen this claim.
🗨️  "This enforcement that was initially intended for a reduction in the health
crisis but is now being viewed as an increase in governmental control." 👉 – This
sentence could be clearer; consider rephrasing to better distinguish between the


## **-- Different Models** (Commented out for now)

In [ ]:
# Gemini
test_feedback_console(
    prompt_type="FewShot",
    model="gemini-1.5-pro-latest",
    requirements_path=reqs_file,
    example_dir=examples_dir,
    target_dir=targets_dir,
    output_dir=outputs_dir,
    output_mode="pretty",
    max_examples=3
)

--- FewShot Feedback for Uncommented_Student_1 ---
📝 Instructor Feedback

--- INLINE FEEDBACK (AT LEAST 4 REQUIRED) ---
👉 “Unfortunately, the year 2019 is known as the year of quarantine.” – This is
incorrect. 2020 was the year lockdowns and quarantines began.  Consider revising
or removing this sentence.
👉 “Although the controversial topic has been popular for some time now,
mandatory mask mandates have been enforced within the past year due to the
coronavirus (COVID-19) pandemic.” –  ^^Mandatory mask mandates^^ is redundant.
Rephrase as ^^mask mandates^^.  Also, while mask-wearing has existed for a long
time (especially in some cultures), mandates for the general public have been
largely limited to the COVID-19 pandemic. Clarify the history for accuracy.
👉 “As a controversial topic, the main sides that divide this issue refer to the
compliance and non-compliance of the mask mandate.” – This could be more
concise.  Consider ^^This controversial issue divides the public into those who


In [ ]:
# # GPT-4.1
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4.1",
#     requirements_path=reqs_file,
#     example_dir=examples_dir,
#     target_dir=targets_dir,
#     output_dir=outputs_dir,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # DeepSeek
# test_feedback_console(
#     prompt_type="FewShot",
#     model="deepseek-chat",
#     requirements_path=reqs_file,
#     example_dir=examples_dir,
#     target_dir=targets_dir,
#     output_dir=outputs_dir,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Llama (different prompt_type)
# test_feedback_console(
#     prompt_type="FewShot-Llama",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     requirements_path=reqs_file,
#     example_dir=examples_dir,
#     target_dir=targets_dir,
#     output_dir=outputs_dir,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Claude
# test_feedback_console(
#     prompt_type="FewShot",
#     model="claude-3-opus-20240229",
#     requirements_path=reqs_file,
#     example_dir=examples_dir,
#     target_dir=targets_dir,
#     output_dir=outputs_dir,
#     output_mode="pretty",
#     max_examples=3
# )